# Complete Visualization Demo: Classifier Model Interpreter

This notebook demonstrates all visualization capabilities including **NEW FEATURES**:
- **Threshold Detection**: Find where feature effects change significantly
- **Auto Segment Discovery**: Find behavioral groups based on SHAP patterns
- **Local Explanations**: Waterfall/Force plots for individual predictions

## What Makes This Package Better Than Native SHAP?

### Key Advantages:

1. **Interactive Plotly visualizations** instead of static matplotlib
2. **Prediction surface visualizations** (contour & 3D) - NOT in SHAP
3. **Threshold detection** - finds non-linear breakpoints automatically
4. **Segment discovery** - finds behavioral groups traditional analytics misses
5. **Local explanation plots** - waterfall charts for individual predictions
6. **Simple, integrated API** - one Interpreter class for everything

## Setup

In [ ]:
import sys
from pathlib import Path

parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from src.core import Interpreter
import warnings
warnings.filterwarnings('ignore')

print("Setup complete")

In [ ]:
# Load the NEW realistic customer conversion data
data_path = Path.cwd().parent / 'data' / 'customer_conversion.csv'
df = pd.read_csv(data_path)

print(f"Dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Conversion rate: {df['converted'].mean():.1%}")
print(f"\nFeatures: {list(df.columns)}")

In [ ]:
# Prepare data - label encode categoricals
cat_cols = ['customer_tier', 'channel', 'device_type', 'region', 'referral_source']
X = df.drop(['customer_id', 'converted'], axis=1).copy()
y = df['converted'].values

label_encoders = {}
for col in cat_cols:
    if col in X.columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        label_encoders[col] = le

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)
model.fit(X_train, y_train)

print(f"Test accuracy: {model.score(X_test, y_test):.3f}")

In [ ]:
# Initialize Interpreter
interp = Interpreter(model, X_test, y_test, config='detailed_analysis')

# Create label mappings for categorical visualizations
tier_labels = {i: label for i, label in enumerate(label_encoders['customer_tier'].classes_)}
channel_labels = {i: label for i, label in enumerate(label_encoders['channel'].classes_)}
device_labels = {i: label for i, label in enumerate(label_encoders['device_type'].classes_)}

print("Interpreter ready")
print(f"\nLabel mappings:")
print(f"  Customer Tier: {tier_labels}")
print(f"  Channel: {channel_labels}")

---
## 1. Global Feature Importance

**What it shows:** Which features matter most overall

In [ ]:
fig = interp.plot_global_importance(top_n=15)
fig.show()

---
## 2. Beeswarm Plot

**What it shows:** Distribution of SHAP values for each feature

In [ ]:
fig = interp.plot_beeswarm(top_n=12)
fig.show()

---
## 3. Feature Dependence Plots

**What it shows:** How feature values affect predictions

In [ ]:
fig = interp.plot_dependence('days_since_login')
fig.show()

In [ ]:
fig = interp.plot_dependence('discount_pct')
fig.show()

In [ ]:
fig = interp.plot_dependence('loyalty_points')
fig.show()

---
## 4. Categorical Dependence Plots

**What it shows:** SHAP distributions for each category (box plots)

In [ ]:
fig = interp.plot_dependence_categorical('customer_tier', value_labels=tier_labels)
fig.show()

In [ ]:
fig = interp.plot_dependence_categorical('channel', value_labels=channel_labels)
fig.show()

---
# NEW FEATURE: Threshold Detection

## 5. Automatic Threshold Detection

**What it does:** Finds values where feature effects CHANGE significantly

**Why it matters:** Traditional analytics shows averages. This finds specific breakpoints:
- "Effect flips at 14 days"
- "Discount only matters above 15%"

**NATIVE SHAP DOESN'T HAVE THIS!**

In [ ]:
# Detect thresholds for days_since_login
threshold_result = interp.detect_thresholds('days_since_login')

print("="*70)
print("THRESHOLD DETECTION: days_since_login")
print("="*70)
print(f"\nMethod: {threshold_result['method']}")
print(f"Samples analyzed: {threshold_result['n_samples']}")

print("\nDetected Thresholds:")
print("-"*70)
for t in threshold_result['thresholds']:
    print(f"\n  Threshold at: {t['value']:.0f} days")
    print(f"    SHAP before: {t['shap_before']:.4f}")
    print(f"    SHAP after:  {t['shap_after']:.4f}")
    print(f"    Effect change: {t['effect_change']:+.4f}")
    print(f"    Confidence: {t['confidence']:.1%}")
    print(f"    -> {t['interpretation']}")

In [ ]:
# Detect thresholds for discount_pct
threshold_result = interp.detect_thresholds('discount_pct')

print("="*70)
print("THRESHOLD DETECTION: discount_pct")
print("="*70)

print("\nDetected Thresholds:")
for t in threshold_result['thresholds']:
    print(f"\n  Threshold at: {t['value']:.0f}%")
    print(f"    Effect change: {t['effect_change']:+.4f}")
    print(f"    -> {t['interpretation']}")

In [ ]:
# Detect thresholds across ALL top features
all_thresholds = interp.detect_all_thresholds(top_n_features=8)

print("ALL DETECTED THRESHOLDS (Top 8 Features)")
print("="*70)
all_thresholds[['feature', 'value', 'effect_change', 'interpretation']].head(10)

---
# NEW FEATURE: Segment Discovery

## 6. Auto Segment Discovery

**What it does:** Finds groups of customers where the MODEL REASONS DIFFERENTLY

**Why it's different from traditional segmentation:**
- Traditional: Groups by demographics (age, tier)
- This: Groups by SHAP patterns (how model explains predictions)

Reveals behavioral segments like:
- "Loyalty Driven" - predictions driven by loyalty points
- "Price Sensitive" - predictions driven by discounts
- "Engagement Dependent" - predictions driven by recency

**NATIVE SHAP DOESN'T HAVE THIS!**

In [ ]:
# Discover behavioral segments
segments = interp.discover_segments(n_segments=4, top_n_features=10)

# Print summary
print(interp.get_segment_summary(segments))

In [ ]:
# Visualize segment profiles
fig = interp.plot_segment_profiles(segments, top_n_features=8)
fig.show()

In [ ]:
# Compare feature importance across segments
fig = interp.plot_segment_comparison(segments, top_n=10)
fig.show()

---
# NEW FEATURE: Local Explanations

## 7. Waterfall & Force Plots

**What it shows:** How a SINGLE prediction was made

- Base value = average model output
- Each feature pushes prediction up or down
- Final prediction = sum of all contributions

Essential for:
- Explaining individual decisions
- Understanding outliers
- Debugging predictions

In [ ]:
# Find a high-probability conversion and a low-probability one
y_proba = model.predict_proba(X_test)[:, 1]

high_prob_idx = np.argmax(y_proba)  # Highest conversion probability
low_prob_idx = np.argmin(y_proba)   # Lowest conversion probability
mid_idx = np.argsort(y_proba)[len(y_proba)//2]  # Middle

print(f"High probability observation: idx={high_prob_idx}, prob={y_proba[high_prob_idx]:.3f}")
print(f"Low probability observation: idx={low_prob_idx}, prob={y_proba[low_prob_idx]:.3f}")
print(f"Mid probability observation: idx={mid_idx}, prob={y_proba[mid_idx]:.3f}")

In [ ]:
# Waterfall plot for HIGH probability prediction
fig = interp.plot_waterfall(high_prob_idx, top_n=10)
fig.show()

print("\nThis customer has HIGH conversion probability because:")
print(interp.explain_observation_text(high_prob_idx, top_n=5))

In [ ]:
# Waterfall plot for LOW probability prediction
fig = interp.plot_waterfall(low_prob_idx, top_n=10)
fig.show()

print("\nThis customer has LOW conversion probability because:")
print(interp.explain_observation_text(low_prob_idx, top_n=5))

In [ ]:
# Force plot (alternative horizontal view)
fig = interp.plot_force(high_prob_idx, top_n=12)
fig.show()

In [ ]:
# Compare multiple observations side by side
fig = interp.plot_multiple_observations([high_prob_idx, mid_idx, low_prob_idx], top_n=8)
fig.show()

---
## 8. Interaction Detection

**What it shows:** Which feature pairs interact most

In [ ]:
interactions = interp.detect_interactions(top_n=15, method='shap_variance')
print("Top 15 Feature Interactions:")
interactions

---
## 9. Prediction Surface Visualizations

**What it shows:** Predicted probability across two features

**Unique to this package** - shows actual predictions (Y), not SHAP values

In [ ]:
# Blocky heatmap: discount × customer tier
fig = interp.plot_interaction_contour(
    'discount_pct', 
    'customer_tier',
    value_labels_2=tier_labels
)
fig.show()

print("\nBusiness insight: How does discount effectiveness vary by customer tier?")

In [ ]:
# Continuous features
fig = interp.plot_interaction_contour('days_since_login', 'engagement_score', n_grid=40)
fig.show()

In [ ]:
# 3D surface plot
fig = interp.plot_interaction_surface_3d('discount_pct', 'loyalty_points', n_grid=25)
fig.show()

---
## 10. Conditional Dependence Plots

**What it shows:** How a feature's effect varies by another feature

- Parallel lines = no interaction
- Diverging lines = strong interaction

In [ ]:
# Does discount effect vary by customer tier?
fig = interp.plot_conditional_dependence(
    'discount_pct', 
    'customer_tier',
    value_labels=tier_labels
)
fig.show()

print("Business question: Does discount effectiveness vary by customer tier?")
print("If lines diverge -> YES, target different tiers with different discounts")

In [ ]:
# Does engagement effect vary by channel?
fig = interp.plot_conditional_dependence(
    'engagement_score', 
    'channel',
    value_labels=channel_labels
)
fig.show()

---
## 11. Model Performance

**Includes:** Confusion matrix, ROC curve, metrics summary, calibration curve

In [ ]:
performance = interp.plot_performance()

performance['metrics_summary'].show()

In [ ]:
performance['confusion_matrix'].show()

In [ ]:
performance['roc_curve'].show()

---
# Summary: New Features in This Package

## NEW Features (Not in SHAP)

1. **Threshold Detection** - Finds non-linear breakpoints automatically
   - "Effect changes at 14 days"
   - "Discount matters above 15%"

2. **Segment Discovery** - Finds behavioral groups based on SHAP patterns
   - Groups by HOW model reasons, not demographics
   - Reveals hidden customer types

3. **Local Explanations** - Waterfall/Force plots for individual predictions
   - Explain specific decisions
   - Compare high vs low predictions

4. **Prediction Surfaces** - 2D/3D visualizations of actual predictions
   - Shows probabilities, not abstract SHAP values
   - Business-friendly interpretation

5. **Conditional Dependence** - Shows interactions visually
   - Parallel lines = no interaction
   - Diverging lines = interaction exists

## Why This Package Beats Native SHAP

- **Interactive Plotly** vs static matplotlib
- **Business-focused** - shows predictions, not just SHAP
- **Discovers insights** - thresholds, segments, interactions
- **Simple API** - one Interpreter class
- **Great for presentations** - interactive, professional visuals